In [ ]:
# !pip install chess

In [1]:
import numpy as np
import pandas as pd
import chess
import chess.pgn
from ChessFunctions import get_encoded_board
from tqdm import tqdm
import dask
import dask.dataframe as dd
import dask.array as da
from dask import delayed

In [2]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:55226 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [3]:
# client.close()

In [4]:
f = open('lichess_elite_2020-06.pgn')

In [5]:
%%time 
my_list = []
Y_labels = []
for game in tqdm(range(2500)):
# while True:
    game = chess.pgn.read_game(f)
    if game is None:
        break  # end of file
    my_list.append(game)

100%|██████████| 2500/2500 [00:06<00:00, 392.00it/s]

CPU times: user 6.21 s, sys: 139 ms, total: 6.34 s
Wall time: 6.38 s


In [6]:
def flatten_board(x):
    return(str(x).replace(' ','').replace('\n',''))

In [7]:
def get_encoded_board(x):
    '''Must pass a flattened board'''
    num_squares = range(64)
    
    #Black Player Feature Vectors
    b_rook  = [1 if x[i]=='r' else 0 for i in num_squares]
    b_night = [1 if x[i]=='n' else 0 for i in num_squares]
    b_bish  = [1 if x[i]=='b' else 0 for i in num_squares]
    b_queen = [1 if x[i]=='q' else 0 for i in num_squares]
    b_king  = [1 if x[i]=='k' else 0 for i in num_squares]
    b_pawn  = [1 if x[i]=='p' else 0 for i in num_squares]
    b_value = [(sum(b_rook)*5 + sum(b_night)*3 + sum(b_bish)*3
           + sum(b_queen)*9 + sum(b_pawn)*1
          )]
    
    #White Player Feature Vectors
    w_rook  = [1 if x[i]=='R' else 0 for i in num_squares]
    w_night = [1 if x[i]=='N' else 0 for i in num_squares]
    w_bish  = [1 if x[i]=='B' else 0 for i in num_squares]
    w_queen = [1 if x[i]=='Q' else 0 for i in num_squares]
    w_king  = [1 if x[i]=='K' else 0 for i in num_squares]
    w_pawn  = [1 if x[i]=='P' else 0 for i in num_squares]
    w_value = [(sum(w_rook)*5 + sum(w_night)*3 + sum(w_bish)*3
           + sum(w_queen)*9 + sum(w_pawn)*1
          )]

    feature_list = (b_rook+b_night+b_bish+b_queen
                +b_king+b_pawn+w_rook+w_night+w_bish
                +w_queen+w_king+w_pawn+b_value+w_value)
    
#     feature_list = dd.from_array(np.append(feature_list,[w_value,b_value]))
    
#     feature_list.extend[w_value,b_value]


#     feature_list = (b_rook+b_night+b_bish+b_queen
#                 +b_king+b_pawn+w_rook+w_night+w_bish
#                 +w_queen+w_king+w_pawn)
    
    return(feature_list)

In [8]:
# def collect_game_states(games):
#     allgame_df = pd.DataFrame()
#     for game in games:
#         board = game.board() 
#         board_state = get_encoded_board(board)
#         for move in game.mainline_moves():
#             board.push(move)
#             board_state = np.row_stack((board_state,get_encoded_board(board)))
#         df = pd.DataFrame(board_state)
#         df['gameid'] = game.headers["LichessURL"]
#         allgame_df = pd.concat([allgame_df,df])
#     ddf = dd.from_pandas(allgame_df,npartitions=1)
#     return ddf

In [29]:
%%time
allgame_df = pd.DataFrame()
for game in tqdm(my_list):
    if game.headers['Termination']=='Normal':
        board = game.board() 
        board_state = [flatten_board(board)]
        for move in game.mainline_moves():
            board.push(move)
            board_state = np.row_stack((board_state,flatten_board(board)))
        df = pd.DataFrame(board_state)
        df['gameid'] = game.headers["LichessURL"]
        df['result'] = (np.where
                        (game.headers["Result"]=="1-0",'w',
                         np.where(game.headers["Result"]=="0-1",'l','d')
                        )
                       )
        df = df[3:]
        allgame_df = pd.concat([allgame_df,df])
    else:
        next

100%|██████████| 2500/2500 [00:17<00:00, 139.18it/s]

CPU times: user 17.4 s, sys: 806 ms, total: 18.2 s
Wall time: 18 s


In [30]:
allgame_df[3:10]

,0,gameid,result
6,rnbqk.nrppp..ppp....p......p.....b.PP.....N......,https://lichess.org/kuUOsOML,d
7,rnbqk.nrppp..ppp....p......pP....b.P......N......,https://lichess.org/kuUOsOML,d
8,rnbqk..rppp.nppp....p......pP....b.P......N......,https://lichess.org/kuUOsOML,d
9,rnbqk..rppp.nppp....p......pP....b.P....P.N......,https://lichess.org/kuUOsOML,d
10,rnbqk..rppp.nppp....p......pP......P....P.b......,https://lichess.org/kuUOsOML,d
11,rnbqk..rppp.nppp....p......pP......P....P.P......,https://lichess.org/kuUOsOML,d
12,rnbqk..rpp..nppp....p.....ppP......P....P.P......,https://lichess.org/kuUOsOML,d


In [31]:
# def func(df):
#     return df['encoded'].apply(pd.Series, 1)
# df.map_partitions(func)

In [32]:
ddf = dd.from_pandas(allgame_df,npartitions=100000)
# del allgame_df

In [33]:
ddf = client.persist(ddf)
# ddf.compute()
# ddf.compute().sort_values(['gameid']).head()

In [34]:
ddf['features'] = ddf[0].apply(get_encoded_board,meta=('I'))

In [35]:
def func(df):
    return df.features.apply(pd.Series, 1).stack().reset_index(level=1, drop=True)

In [36]:
# da_X = da.array(ddf.map_partitions(func))
# da_X.compute().shape

In [37]:
feature_array = np.array(ddf.map_partitions(func).compute()).reshape(-1,770)
feature_array.shape

(138986, 770)

In [38]:
# ddf['encoded']=ddf[0].apply(get_encoded_board,meta=('I'))
# encoded_array = da.concatenate(ddf[0].apply(get_encoded_board,meta=('I')))
# ddf_X = dd.DataFrame(encoded_array, name='test',divisions=4,meta=("DF")) 
# test = ddf.map_partitions(func)
# ddf = ddf.map_partitions(lambda df: df['encoded'].apply(pd.Series,1))

In [39]:
# %%time
# pandas_playground = pd.DataFrame(ddf.compute())
# test = pandas_playground['encoded'].apply(pd.Series)
# test

In [40]:
# %%time
# game_state_dictionary = {}
# Y_labels = []
# i = 0
# for game in tqdm(my_list):
#     board = game.board()
#     board_state = get_encoded_board(board)
#     # Win
#     if game.headers['Result'] == "1-0":
#         Y_labels.append("w")
#     # Lose
#     elif game.headers['Result'] == "0-1":
#         Y_labels.append('l')
#     # Draw
#     else:
#         Y_labels.append('d')
#     for move in game.mainline_moves():
#         board.push(move)
#         board_state = np.row_stack((board_state,get_encoded_board(board)))
#         # Win
#         if game.headers['Result'] == "1-0":
#             Y_labels.append('w')
#         # Lose
#         elif game.headers['Result'] == "0-1":
#             Y_labels.append('l')
#         # Draw
#         else:
#             Y_labels.append('d')
#     game_state_dictionary[i] = board_state
#     i = i+1

In [41]:
# game_state_dictionary[999]

In [42]:
# len(Y_labels)

In [43]:
# X = np.row_stack([game_state_dictionary[x] for x in sorted(game_state_dictionary)])
# X.shape

X = feature_array
Y_labels = allgame_df['result']
print("X: " + str(X.shape))
print("Y: " + str(Y_labels.shape))

X: (138986, 770)
Y: (138986,)


In [44]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(
    X, Y_labels, stratify=Y_labels, train_size = 0.9
    ,random_state=1
)

CPU times: user 276 ms, sys: 270 ms, total: 546 ms
Wall time: 542 ms


In [45]:
X_train.shape, X_test.shape

((125087, 770), (13899, 770))

In [ ]:
param_grid = {'hidden_layer_sizes':[(300,200,100,50), (300,200,100,50,25)]
              ,'tol':[0.00001]
              ,'max_iter':[600]
              ,'learning_rate_init':[0.0001]
             }
mlp = MLPClassifier(early_stopping=True)
clf = GridSearchCV(mlp, param_grid, n_jobs=-1)
# clf = MLPClassifier(hidden_layer_sizes=(400),random_state=1
#                     ,early_stopping=True,tol=.00001,max_iter=300).fit(X_train, y_train)
clf.fit(X_train, y_train)

In [ ]:
print(clf.score(X_test, y_test))
clf.best_estimator_

In [ ]:
# clf.predict(X_test)

In [ ]:
from joblib import dump, load
dump(clf, 'filename.joblib')  

In [ ]:
clf = load('filename.joblib') 

In [ ]:
import torch

In [ ]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

In [ ]:
model = Feedforward(770, 100)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
import tensorflow as tf
X_test_tensor = tf.convert_to_tensor(X_test)

In [ ]:
model.eval()
y_pred = model(X_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())
